<a href="https://colab.research.google.com/github/kcanmersin/Google-Colab-Notebooks/blob/main/food.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import kaggle
import gdown

def download_kaggle_dataset_to_drive(dataset_name, drive_folder_path):
    # Create a folder in Google Drive to store the downloaded dataset
    if not os.path.exists(drive_folder_path):
        os.makedirs(drive_folder_path)

    # Set the path where you want to download the dataset in the drive_folder_path
    download_path = os.path.join(drive_folder_path, dataset_name)

    # Use the Kaggle API to download the dataset files
    kaggle.api.dataset_download_files(dataset_name, path=download_path, unzip=True, quiet=True)

    # List the downloaded files
    downloaded_files = os.listdir(download_path)
    print(f'Downloaded {len(downloaded_files)} files from the dataset: {dataset_name}')

# Example usage:
dataset_name = 'kmader/food41'
drive_folder_path = '/content/drive/MyDrive/KaggleDatasets'
download_kaggle_dataset_to_drive(dataset_name, drive_folder_path)
csv_file_path = os.path.join(drive_folder_path, dataset_name)


In [ ]:
file_path = os.path.join(drive_folder_path, dataset_name)

In [ ]:
file_path

In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
import os.path

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split

import tensorflow as tf

from sklearn.metrics import confusion_matrix,classification_report

In [ ]:
image_dir = Path(file_path+"/images")

In [ ]:
image_dir

### Creating Dataframe


In [ ]:
filepaths=list(image_dir.glob(r"**/*.jpg"))

In [ ]:
labels=list(map(lambda x:os.path.split(os.path.split(x)[0])[1],filepaths))

In [ ]:
len(labels)

In [ ]:
filepaths = pd.Series(filepaths,name="Filepath").astype(str)
labels = pd.Series(labels,name="Label")


In [ ]:
images = pd.concat([filepaths,labels],axis=1)

In [ ]:
images.head()

In [ ]:
category_samples=[]

for category in images["Label"].unique():
  category_slice = images.query("Label == @category")
  category_samples.append(category_slice.sample(300,random_state=2))
image_df = pd.concat(category_samples,axis=0).sample(frac=1.0,random_state=2).reset_index(drop=True)

In [ ]:
image_df

#Train test Split



In [ ]:
train_df, test_df = train_test_split(image_df,train_size=0.7,shuffle=True,random_state=2)

In [ ]:
import tensorflow as tf

train_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input,
    validation_split=0.2
)

test_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input
)


In [ ]:
train_images=train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col="Filepath",
    y_col="Label",
    target_size=(224,224),
    color_mode="rgb",
    class_mode="categorical",
    batch_size=32,
    shuffle=True,
    seed=42,
    subset="training"
)
val_images=train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col="Filepath",
    y_col="Label",
    target_size=(224,224),
    color_mode="rgb",
    class_mode="categorical",
    batch_size=32,
    shuffle=True,
    seed=42,
    subset="validation"
)
test_images=test_generator.flow_from_dataframe(
    dataframe=test_df,
    x_col="Filepath",
    y_col="Label",
    target_size=(224,224),
    color_mode="rgb",
    class_mode="categorical",
    batch_size=32,
    shuffle=False
)


#Model MobileNetV2



In [ ]:
pretrained_model=tf.keras.applications.MobileNetV2(
    input_shape=(224,224,3),
    include_top=False,
    weights="imagenet",
    pooling="avg"
)
pretrained_model.trainable=False

In [ ]:
 inputs=pretrained_model.input

 x=tf.keras.layers.Dense(129,activation="relu")(pretrained_model.output)
 x=tf.keras.layers.Dense(129,activation="relu")(x)

outputs= tf.keras.layers.Dense(101,activation="softmax")(x)
model = tf.keras.Model(inputs,outputs)

#Training

In [ ]:
model.compile(
    optimizer="adam",
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)
history=model.fit(
    train_images,
    validation_data=val_images,
    epochs=100,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor="val_loss",
            patience=10,
            restore_best_weights=True
        )
    ]
)

#Result 1


In [ ]:
results = model.evaluate(test_images,verbose=0)
print("Test Accuracy: {:.2f}%".format(results[1]*100))

In [ ]:
predictions=np.argmax(model.predict(test_images),axis=1)

cm = confusion_matrix(test_images.labels,predictions)
clr = classification_report(test_images.labels,predictions,target_names=test_images.class_indices)

In [ ]:
plt.figure(figsize=(30,30))
sns.heatmap(cm,annot=True,fmt="g",vmin=0,cmap="Blues",cbar=False,rotation=90)
plt.xticks(ticks=np.arange(101)+0.5,labels=test_images.class_indices)
plt.xlabel("predicted")
plt.ylabel("actual")
plt.title("confusion matrix")
plt.show()


In [ ]:
print(clr)

#model 2


In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50

pretrained_model = ResNet50(
    input_shape=(224, 224, 3),
    include_top=False,
    weights="imagenet",
    pooling="avg"
)
pretrained_model.trainable = False

inputs = pretrained_model.input
x = tf.keras.layers.Dense(129, activation="relu")(pretrained_model.output)
x = tf.keras.layers.Dense(129, activation="relu")(x)

outputs = tf.keras.layers.Dense(101, activation="softmax")(x)
model = tf.keras.Model(inputs, outputs)
model.compile(
    optimizer="adam",
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

history = model.fit(
    train_images,
    validation_data=val_images,
    epochs=20,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor="val_loss",
            patience=10,
            restore_best_weights=True
        )
    ]
)


In [ ]:
results = model.evaluate(test_images,verbose=0)
print("Test Accuracy: {:.2f}%".format(results[1]*100))

In [ ]:
predictions=np.argmax(model.predict(test_images),axis=1)

cm = confusion_matrix(test_images.labels,predictions)
clr = classification_report(test_images.labels,predictions,target_names=test_images.class_indices)

In [ ]:
plt.figure(figsize=(30,30))
sns.heatmap(cm,annot=True,fmt="g",vmin=0,cmap="Blues",cbar=False,rotation=90)
plt.xticks(ticks=np.arange(101)+0.5,labels=test_images.class_indices)
plt.xlabel("predicted")
plt.ylabel("actual")
plt.title("confusion matrix")
plt.show()
